In [3]:
import openai
import json
import os 

from dotenv import load_dotenv
from polywrap_client import PolywrapClient
from typing import cast

from pathlib import Path
from polywrap_client import PolywrapClient, ClientConfig
from polywrap_core import Uri, InvokerOptions, UriPackageOrWrapper, UriResolver
from polywrap_client_config_builder import PolywrapClientConfigBuilder
from polywrap_uri_resolvers import FsUriResolver,SimpleFileReader, StaticResolver, RecursiveResolver
from polywrap_uri_resolvers import UriResolverAggregator
from polywrap_http_plugin import http_plugin


import asyncio

import nest_asyncio

nest_asyncio.apply()

load_dotenv()

ipfs_wrapper_path = Path("fs//Users/robertohenriquez/pycode/polywrap/hackathon/Auto-GPT/autogpt/auto_gpt_workspace/wrappers/ipfs-http-client")


resolver = RecursiveResolver(
        UriResolverAggregator(
            [
                cast(UriResolver, FsUriResolver(file_reader=SimpleFileReader())),
                cast(UriResolver, StaticResolver({Uri("wrap://ens/wraps.eth:http@1.1.0", ipfs_wrapper_path): http_plugin()})),
            ]
        )
    )
config = ClientConfig(resolver=resolver)
client = PolywrapClient(config)

openai.api_key = os.getenv("OPENAI_API_KEY")



def polywrap_bignumber(base_number, factor):
    """Multiplies a base number by a factor to get a new number"""
    uri = Uri.from_str(
        f'fs//Users/robertohenriquez/pycode/cloud/AGI/Auto-GPT/autogpt/auto_gpt_workspace/wrappers/bignumber'
    )
    args = {
        "arg1": str(base_number),  # The base number
        "obj": {
            "prop1": str(factor),  # multiply the base number by this factor
        },
    }
    options: InvokerOptions[UriPackageOrWrapper] = InvokerOptions(
        uri=uri, method="method", args=args, encode_result=False
    )
    print(asyncio.run(client.invoke(options)))
    result = asyncio.run(client.invoke(options))
    return f"the result is {result}"


In [11]:
import json
import openai

functions = [
    {
        "name": "polywrap_bignumber",
        "description": "Use this function every time you need to make a multiplication",
        "parameters": {
            "type": "object",
            "properties": {
                "base_number": {
                    "type": "number",
                    "description": "The base number",
                },
                "factor": {
                    "type": "number",
                    "description": "The factor to multiply the base number",
                },
            },
            "required": ["base_number", "factor"],
        },
    },
    {
        "name": "fetch_tool_library",
        "description": "Fetches information about a specific tool from the tool library, given the tool name.",
        "parameters": {
            "type": "object",
            "properties": {
                "tool_name": {
                    "type": "string",
                    "description": "The name of the tool to fetch",
                },
            },
            "required": ["tool_name"],
        },
    },
]

question = "Find what tools are available for multiplication of big numbers"


def run_conversation(question, functions):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": question}],
        functions=functions,
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Check if the model wants to call a function
    if message.get("function_call"):
        print("-> Using a function call now")
        function_name = message["function_call"]["name"]
        function_args_string = message["function_call"]["arguments"]
        function_args = json.loads(function_args_string)  # Parse the string into a dictionary

        # Find the function based on its name
        function = next((func for func in functions if func["name"] == function_name), None)

        if function:
            # Call the function dynamically
            function_response = globals()[function_name](**function_args)

            # Send the function call and response to the model
            second_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0613",
                messages=[
                    {"role": "user", "content": "What is the result of the initial question?"},
                    message,
                    {
                        "role": "function",
                        "name": function_name,
                        "content": function_response,
                    },
                ],
            )
            return second_response
        else:
            raise Exception(f"Function '{function_name}' not found")
    else:
        raise Exception("The wrapper was not invoked")


def polywrap_bignumber(base_number, factor):
    # Replace this with your implementation of the polywrap_bignumber function
    result = base_number * factor
    return result

def fetch_tool_library(tool_name):
    return """[
    "wrap/ipfs",
    "wrap/http",
    "wrap/ens",
    "wrap/ethers"
]"""


print(run_conversation(question, functions))


-> Using a function call now
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The result of the initial question is a list of tools that can be used for multiplication. These tools include \"wrap/ipfs\", \"wrap/http\", \"wrap/ens\", and \"wrap/ethers\".",
        "role": "assistant"
      }
    }
  ],
  "created": 1687348387,
  "id": "chatcmpl-7TqgViFsWIA1ulGRUZGCDYKRbRnUY",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 42,
    "prompt_tokens": 69,
    "total_tokens": 111
  }
}
